In [1]:
import pandas as pd
from openpyxl import Workbook
import gradio as gr

# Nama file dataset (Pastikan file ini ada di folder yang sama)
file_path = "dataset_50_karyawan_konflik_backtracking.xlsx"

try:
    df_karyawan = pd.read_excel(file_path, sheet_name='Karyawan')
    df_shift = pd.read_excel(file_path, sheet_name='Shift')
    df_larangan = pd.read_excel(file_path, sheet_name='Larangan_Shift')
    
    karyawan_list = df_karyawan.to_dict('records')
    shift_list = df_shift.to_dict('records')
    
    # Mapping kategori shift ke rentang waktu (Akan ditampilkan di kolom 'Waktu')
    jam_range_map = {
        "Pagi": "08.00 - 14.00",
        "Siang": "14.00 - 20.00",
        "Malam": "20.00 - 02.00"
    }
    
    # Set larangan untuk validasi cepat
    larangan_set = set(zip(df_larangan['ID_Karyawan'], df_larangan['Hari'], df_larangan['Jam']))
    
    print("Dataset Berhasil Dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset Berhasil Dimuat.


In [6]:
# Inisialisasi variabel status
jadwal_final = []
jumlah_shift_karyawan = {k['ID_Karyawan']: 0 for k in karyawan_list}
karyawan_per_hari = {}

def is_safe(karyawan, shift):
    # --- KONSEP BOBOT (w) ---
    # Bobot maksimal (W) tiap karyawan diambil dari kolom 'Max_Shift'
    W_maksimal = karyawan['Max_Shift']
    w_sekarang = jumlah_shift_karyawan[karyawan['ID_Karyawan']]
    
    # Periksa apakah penambahan bobot (w=1) melampaui kapasitas W
    if w_sekarang >= W_maksimal:
        return False
    
    # --- VALIDASI ATURAN LAIN ---
    if karyawan['Skill'] != shift['Posisi']:
        return False
    
    if (karyawan['ID_Karyawan'], shift['Hari'], shift['Jam']) in larangan_set:
        return False
    
    hari = shift['Hari']
    if hari not in karyawan_per_hari:
        karyawan_per_hari[hari] = set()
    
    if karyawan['ID_Karyawan'] in karyawan_per_hari[hari]:
        return False
        
    return True

In [8]:
def solve_backtracking(shift_idx):
    # TARGET OPTIMASI: Tercapainya total Profit (P) = Total Slot Shift (105)
    if shift_idx == len(shift_list):
        return True
    
    current_shift = shift_list[shift_idx]
    
    for k in karyawan_list:
        if is_safe(k, current_shift):
            # LANGKAH FORWARD: 
            # - Menugaskan karyawan memberikan Profit p = 1
            # - Menugaskan karyawan mengonsumsi Bobot w = 1
            w = 1
            
            jadwal_final.append({
                'ID_Karyawan': k['ID_Karyawan'],
                'Nama': k['Nama'],
                'Hari': current_shift['Hari'],
                'Waktu': jam_range_map.get(current_shift['Jam'], ""), # Penamaan kolom baru
                'Posisi': current_shift['Posisi']
            })
            
            # Update status beban kerja (Bobot) dan kehadiran harian
            jumlah_shift_karyawan[k['ID_Karyawan']] += w
            karyawan_per_hari[current_shift['Hari']].add(k['ID_Karyawan'])
            
            # REKURSI: Mencari profit untuk slot berikutnya
            if solve_backtracking(shift_idx + 1):
                return True
            
            # LANGKAH BACKTRACK: 
            # Membatalkan profit dan mengembalikan bobot jika jalur buntu
            jadwal_final.pop()
            jumlah_shift_karyawan[k['ID_Karyawan']] -= w
            karyawan_per_hari[current_shift['Hari']].remove(k['ID_Karyawan'])
            
    return False

In [9]:
def jalankan_penjadwalan():
    global jadwal_final, jumlah_shift_karyawan, karyawan_per_hari
    # Reset semua variabel status
    jadwal_final = []
    jumlah_shift_karyawan = {k['ID_Karyawan']: 0 for k in karyawan_list}
    karyawan_per_hari = {}
    
    if solve_backtracking(0):
        df_hasil = pd.DataFrame(jadwal_final)
        df_hasil.to_excel("Hasil_Jadwal_Final_Waktu.xlsx", index=False)
        return df_hasil
    else:
        return None

In [10]:
def dashboard():
    with gr.Blocks(title="Aplikasi Penjadwalan") as demo:
        gr.Markdown("# 📅 Penjadwalan Karyawan Berbasis Backtracking")
        gr.Markdown("Dashboard interaktif untuk menghasilkan jadwal shift tanpa konflik.")
        
        btn = gr.Button("Generate Jadwal", variant="primary")
        
        # Output dataframe hanya menampilkan kolom yang telah disesuaikan
        output = gr.Dataframe(
            headers=["ID_Karyawan", "Nama", "Hari", "Waktu", "Posisi"],
            interactive=False
        )
        
        btn.click(fn=jalankan_penjadwalan, outputs=output)
    
    demo.launch()

# Menjalankan aplikasi
dashboard()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
